In [67]:
import pandas as pd

In [68]:
df = pd.read_csv("../../model_2/data/raw/sofifa_players.csv", index_col=False)

/var/folders/0_/_3dg0b615p38ptjrygv5db6h0000gn/T/ipykernel_86457/2871305303.py:1: DtypeWarning: Columns (3,4,13,14,19,20,21,22,23,25,26,27,28,29,31,32,33,34,35,37,38,39,40,41,43,44,45,46,47,48,50,51,52,57,58,69,70,71,73) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../../model_2/data/raw/sofifa_players.csv", index_col=False)


In [69]:
df = df[df['season'] == 2023].drop(["Unnamed: 0", 'Traits', 'Traits.1',
       'PlayStyles', 'PlayStyles +',  'Unnamed: 76', "Acceleration type", "Loan date end"], axis=1)

In [70]:
df


,Name,Age,Overall rating,Potential,Team & Contract,ID,Height,Weight,foot,Best overall,Best position,Growth,Joined,Value,Wage,Release clause,Total attacking,Crossing,Finishing,Heading accuracy,Short passing,Volleys,Total skill,Dribbling,Curve,FK Accuracy,Long passing,Ball control,Total movement,Acceleration,Sprint speed,Agility,Reactions,Balance,Total power,Shot power,Jumping,Stamina,Strength,Long shots,Total mentality,Aggression,Interceptions,Attack position,Vision,Penalties,Composure,Total defending,Defensive awareness,Standing tackle,GK Diving,Total stats,Base stats,Weak foot,Skill moves,Attacking work rate,Defensive work rate,International reputation,Body type,Real face,Pace / Diving,Shooting / Handling,Passing / Kicking,Dribbling / Reflexes,Defending / Pace,Physical / Positioning,Number of playstyles,Club position,Club kit number,season,season_code
5631,K. De BruyneCMCAM,31,91,91,Manchester City2015 ~ 2025,192985,"181cm 5'11""",75kg 165lbs,Right,91,CM,0,"Aug 30, 2015",€107.5M,€350K,€198.9M,410,94,85,55,93,83,441,86,89,83,93,90,393,75,73,76,91,78,409,92,63,89,74,91,405,75,65,88,94,83,88,185,66,66,15,2299,481,5,4,High,Medium,4,Unique,Yes,74,88,93,86,63,77,7,RCM,17,2023,230054
5632,L. MessiRW,35,91,91,Paris Saint-Germain2021 ~ 2023,158023,"169cm 5'7""",67kg 148lbs,Left,92,CAM,0,"Aug 10, 2021",€54M,€195K,€99.9M,423,84,90,70,91,88,464,95,93,93,90,93,441,87,76,91,92,95,383,86,68,70,68,91,346,44,40,93,94,75,96,79,20,35,6,2190,452,4,4,Low,Low,5,Unique,Yes,81,89,90,94,34,64,7,RS,30,2023,230054
5633,K. MbappéSTLW,23,91,95,Paris Saint-Germain2018 ~ 2024,231747,"182cm 6'0""",75kg 165lbs,Right,92,ST,4,"Jul 1, 2018",€190.5M,€230K,€366.7M,411,78,93,72,85,83,404,93,80,69,71,91,461,97,97,93,93,81,410,88,77,87,76,82,361,64,38,92,83,84,88,92,26,34,13,2181,470,4,5,High,Low,4,Unique,Yes,97,89,80,92,36,76,5,LS,7,2023,230054
5634,R. LewandowskiST,33,91,91,FC Barcelona2022 ~ 2025,188545,"185cm 6'1""",81kg 179lbs,Right,91,ST,0,"Jul 18, 2022",€84M,€420K,€172.2M,429,71,94,91,84,89,408,85,79,85,70,89,403,76,75,77,93,82,423,91,85,76,87,84,395,81,49,94,81,90,88,96,35,42,15,2205,458,4,4,High,Medium,5,Unique,Yes,75,91,79,86,44,83,5,ST,9,2023,230054
5635,K. BenzemaCFST,34,91,91,Real Madrid2009 ~ 2023,165153,"185cm 6'1""",81kg 179lbs,Right,91,CF,0,"Jul 9, 2009",€64M,€450K,€131.2M,433,75,91,90,89,88,409,87,82,73,76,91,398,78,79,77,92,72,411,87,79,82,82,81,366,63,39,92,90,82,90,85,43,24,13,2143,454,4,4,Medium,Medium,4,Normal (170-185),Yes,79,88,83,87,39,78,5,CF,9,2023,230054
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8543,F. BrancoliniGK,20,53,65,Lecce2022 ~ 2027,257580,"192cm 6'4""",78kg 172lbs,Right,53,GK,12,"Jul 1, 2022",€200K,€1K,€400K,68,14,7,10,29,8,68,5,14,11,27,11,154,21,32,30,45,26,170,45,37,22,59,7,99,27,8,8,38,18,25,26,5,10,48,853,293,1,1,Medium,Medium,1,Lean (185+),No,48,54,60,51,25,55,0,RES,21,2023,230054
8544,A. SorrentinoGK,17,53,68,US Salernitana 19192021 ~ 2027,271505,"183cm 6'0""",80kg 176lbs,Right,53,GK,15,"Jul 1, 2021",€190K,€500,€461K,53,11,8,11,17,6,69,9,14,10,20,16,177,27,31,33,43,43,162,37,51,19,46,9,78,20,8,5,30,15,39,30,6,10,55,832,292,2,1,Medium,Medium,1,Normal (170-185),No,55,50,49,58,29,51,0,RES,72,2023,230054
8545,C. SlickerGK,19,52,69,Manchester City2020 ~ 2024,264645,"191cm 6'3""",82kg 181lbs,Right,52,GK,17,"Jul 1, 2020",€180K,€4K,€446K,56,12,8,10,17,9,77,8,10,12,29,18,136,30,22,21,41,22,156,40,33,24,51,8,97,27,10,4,37,19,36,31,8,12,48,815,289,2,1,Medium,Medium,1,Normal (185+),No,48,60,53,56,27,45,0,RES,84,2023,230054
8546,D. CiezkowskiGK,21,51,58,Cremonese2021 ~ 2023,268636,"191cm 6'3""",80kg 176lbs,Left,51,GK,7,"Jul 30, 2021",€100K,€950,€200K,63,13,8,10,24,8,65,10,14,13,18,10,153,22,27,33,44,27,155,37,31,18,61,8,84,17,9,5,37,16,30,32,6,12,48,806,278,2,1,Medium,Medium,1,Normal (185+),N

In [ ]:
import re


In [ ]:
def _clean_sofifa_data( df):
    df.columns = df.columns.str.replace(" / ", "_").str.replace(" & ", "_").str.replace(" ", "_")

    df['height(cm)'] = df["height"].str.split("cm").str[0].astype(int)
    df['weight(kg)'] = df["weight"].str.split("kg").str[0].astype(int) 

    df['foot'] = df['foot'].map({"Left": 1, "Right": 2})

def parse_monetary_value(row): 
    row = row.split('€')[1]
    if "M" in row:
        value = float(row.replace("M", "")) * 1_000_000
    elif "K" in row:
        value = float(row.replace("K", "")) * 1_000
    else:
        value = float(row)

    return value

def extract_team_name(row):
    """Extract team name from 'Team & Contract' column by splitting on first 4-digit pattern"""
    match = re.search(r'\d{4}', str(row))
    if match:
        # Get position of first 4-digit pattern
        position = match.start()
        # Extract everything before it and strip whitespace
        team_name = row[:position].strip()
        return team_name
    else:
        # If no 4-digit pattern found, return original value
        return row

def extract_primary_position(row):
    if "," in row:
        row = row.split(",")[0]
    
    return row.strip()

def extract_name_from_position(row):
    pattern = r'[A-Z]{2,}'
    match = re.search(pattern, row)
    if match:
        location = match.start()
        name = row[:location].strip()
        
        return name
    else:
        return "HELLO"

df.drop_duplicates(inplace=True)
df.columns = df.columns.str.strip().str.lower()

df['name'] = df['name'].apply(extract_name_from_position)
df["name"] = df['name'].str.lower()      

_clean_sofifa_data(df)
df['value(€)'] = df['value'].apply(parse_monetary_value)
df['wage(€)'] = df['wage'].apply(parse_monetary_value)
df['team'] = df['team_contract'].apply(extract_team_name)
df.drop(['team_contract', 'height', 'weight', 'real_face', 'body_type', 'value', 'wage'], axis=1, inplace=True)


In [73]:
# max column number
pd.set_option('display.max_columns', None)